# Implementing a bot to discuss and aggregate reviews

In [1]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Loading source DB

In [2]:
import pandas as pd
import os

df = pd.read_csv("data\kag_comb_clustered.csv")

df.columns

<>:4: SyntaxWarning: invalid escape sequence '\k'
<>:4: SyntaxWarning: invalid escape sequence '\k'
C:\Users\aurel\AppData\Local\Temp\ipykernel_27232\2194879890.py:4: SyntaxWarning: invalid escape sequence '\k'
  df = pd.read_csv("data\kag_comb_clustered.csv")


Index(['brand', 'categories', 'id', 'manufacturer', 'name',
       'reviews.doRecommend', 'reviews.id', 'reviews.rating', 'reviews.text',
       'reviews.title', 'reviews.username', 'sentiment', 'clean title',
       'clean review', 'title sentiment', 'review sentiment',
       'title sentiment details', 'review sentiment details',
       'merged PySent sentiment', 'PySent correct', 'classifier src 0',
       'classifier src 1', 'classifier src 2', 'classifier src 3',
       'classifier lemma src 0', 'classifier lemma src 1',
       'classifier lemma src 2', 'classifier lemma src 3', 'cluster',
       'cluster_name'],
      dtype='object')

## Selecting model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
model = 'mistralai/Mistral-Nemo-Instruct-2407'

# Load path from the environment variable
env_lib1 = os.getenv("ENV_IH1")
dotenv_path = Path(env_lib1)
load_dotenv(dotenv_path)
MISTRAL_KEY = os.getenv("MISTRAL_SUMMARIZER")

NameError: name 'Path' is not defined